# Data Processing

In [1]:
import re 
from pathlib import Path
import os
import pandas as pd

In [2]:
genres = ['Computers', 'Business', 'Health', 'Romance', 'Horror']
path = 'Clasif_data'

In [4]:
text_finetuning = ''
for i, genre in enumerate(genres):
    for r, d, f in os.walk(path + '/' + genre): #I walk through the folders and take files one by one
        num_of_books=len(f)
        for j, file in enumerate(f):
            with open(path + '/' + genre + '/' + file, 'r') as book:
                 lines=book.readlines()
            lenght_book = len(lines)
            book_portion = round(len(lines)/num_of_books+2)#indicates the part of the book to take the page
            paginas = ''
            for k in range(j+1):#j indicates the number of pages I want on that book. 
                paginas = paginas + ''.join(lines[book_portion*(k+1):book_portion*(k+1)+60])
            text_finetuning = paginas + text_finetuning #I include all the pages in this file.
            with open(path + '/' + genre + '/' + file, 'w') as book:
                book.write(paginas)
            with open(path + '/text_finetuning_books.txt' , 'w') as book:
                book.write(text_finetuning)


In [ ]:
#I remove all the blank lines
with open(path + '/text_finetuning_books.txt') as infile, open(path + '/text_finetuning_books2.txt', 'w') as outfile:
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output

In [127]:
train_df = pd.DataFrame(columns=['Class', 'Text'])
dev_df = pd.DataFrame(columns=['Class', 'Text'])
test_df = pd.DataFrame(columns=['Class', 'Text','Book'])

for i, genre in enumerate(genres):
    for r, d, f in os.walk(path + '/' + genre):
        num_of_books=len(f)
        train_dev_limit = round(num_of_books*0.6) #until this limit is the train set
        dev_test_limit = round(num_of_books*0.8)#from the previous limit until this one is the dev set.
        for j, file in enumerate(f):
            with open(path + '/' + genre + '/' + file, 'r') as book:
                 lines = book.readlines()
            lenght_book = len(lines)
            # I calculated that 30 lines are half page. I divided by 32 to leave some margin to the loop to be able
            # skip the first and last page
            num_half_pages = round(lenght_book/32)
            
            if j < train_dev_limit:#I check if the book belongs to the train set
                for k in range(num_half_pages):
                    train_df.loc[len(train_df)] = [i, ''.join(lines[(k+2)*30:(k+3)*30])]
                  
            elif j < dev_test_limit:#I check if the book belongs to the dev set
                for k in range(num_half_pages):
                    dev_df.loc[len(dev_df)] = [i, ''.join(lines[(k+2)*30:(k+3)*30])]
                   
            else: #the book belongs to the test set. Depending on the lenght, I will test a number of paragraphs.
                if len(lines) > 19*60:
                    num_paragraph_tested = round(((len(lines)/60))/5)#1/5 of the pages will be tested
                elif len(lines) > 14*60:
                    num_paragraph_tested = 4 #around 4 pages will be tested
                elif len(lines) > 8*60: 
                    num_paragraph_tested = 3 #around 3 pages will be tested
                elif len(lines) > 2*60: 
                    num_paragraph_tested = 2 #around 2 pages will be tested
                else:
                    num_paragraph_tested = 1
                book_portion = round(len(lines)/(num_paragraph_tested+1))#index that points the paragraph to pick
                
                #creates a dataframe with paragraphs of 30 lines equally picked from the whole book
                for k in range(num_paragraph_tested): 
                    test_df.loc[len(test_df)] = [i, ''.join(lines[(k+1)*book_portion:((k+1)*book_portion) + 30]), j]

In [131]:
print(len(train_df))
print(len(dev_df))
print(len(test_df))

46597
15221
2183


In [132]:
#We clean short lines
train_df = train_df[train_df.Text.str.len() > 20]
dev_df = dev_df[dev_df.Text.str.len() > 20]
test_df = test_df[test_df.Text.str.len() > 20]

In [133]:
print(len(train_df))
print(len(dev_df))
print(len(test_df))

46573
15216
2182


In [136]:
train_df.head()

,Class,Text
0,0,cooperation. Although it may have disappointed...
1,0,people are allowed to participate actively in ...
2,0,"strategies sell more products, earn more votes..."
3,0,"conflict than were available, particularly to ..."
4,0,While it may not provide us with a template fo...


In [137]:
test_df.head()

,Class,Text,Book
0,0,Novembre 2000\n\n= Mise en ligne de la version...,56
1,0,= Mail-romans\n\nDernier-né de la cyber-littér...,56
2,0,\n\n[5.1. Numérisation: mode texte et mode ima...,56
3,0,"Ce logiciel étant téléchargeable gratuitement,...",56
4,0,"\n\nConsidéré par beaucoup comme transitoire, ...",56
